# yolo-preprocessed

In [1]:
%%capture
!pip install ultralytics wandb scikit-learn matplotlib
 

In [2]:
import os
from pathlib import Path
import shutil
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
import wandb
from sklearn.metrics import f1_score, confusion_matrix
import sys
sys.path.append('.')

In [3]:
run_name = "yolo11s-cls-pmg-512"
model_version = 'yolo11s-cls.pt'
data_dir = "./ODIR-2019/YOLO/processed_512g_merged"
project_name = "odir-2019-yolo"
saved_model_name = 'yolo11s-cls-odir-2019-pmg.pt'
result_folder = 'ODIR-2019/results/'
final_model_path = os.path.join(result_folder, saved_model_name)
image_size = 512
batch_size = 16
epochs = 50
workers_count = 4
patience=30

## Train

In [4]:
run_id=wandb.init(
    project=project_name,
    name=run_name,
    config={
        "img_size": image_size,
        "batch_size": batch_size,
        "epochs": epochs,
        "model": model_version,
        "patience": patience,
    },
    resume="allow"
)

wandb: Currently logged in as: raymond-samalo (samalo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
model = YOLO(model_version)

In [6]:
# ---------------------------
# F1 Tracking Lists
# ---------------------------
train_f1_list = []
val_f1_list = []

# 6️⃣ Custom macro-F1 calculation
def compute_macro_f1(model, folder):
    classes = sorted([d.name for d in Path(folder).iterdir() if d.is_dir()])
    class_to_idx = {c:i for i,c in enumerate(classes)}
    y_true, y_pred = [], []

    for cls in classes:
        for img_path in (Path(folder)/cls).glob("*"):
            y_true.append(class_to_idx[cls])
            r = model.predict(str(img_path))[0]
            probs = r.probs.data.cpu().numpy()[0]
            y_pred.append(probs.argmax())

    macro_f1 = f1_score(y_true, y_pred, average="macro")
    return macro_f1, classes, y_true, y_pred

In [7]:
compute_macro_f1(model, os.path.join(data_dir, "val")) # Test the function works


image 1/1 /home/ray/Projects/ITI123-GenerativeAIDeepLearningProject/ODIR-2019/YOLO/processed_512g_merged/val/ageing/260_left.jpg: 224x224 jellyfish 0.45, bubble 0.35, balloon 0.08, Petri_dish 0.03, vase 0.01, 2.3ms
Speed: 3.3ms preprocess, 2.3ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /home/ray/Projects/ITI123-GenerativeAIDeepLearningProject/ODIR-2019/YOLO/processed_512g_merged/val/ageing/514_left.jpg: 224x224 gong 0.52, shield 0.08, umbrella 0.05, lampshade 0.05, chime 0.04, 2.3ms
Speed: 2.8ms preprocess, 2.3ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /home/ray/Projects/ITI123-GenerativeAIDeepLearningProject/ODIR-2019/YOLO/processed_512g_merged/val/ageing/808_right.jpg: 224x224 lampshade 0.76, toilet_seat 0.05, balloon 0.04, gong 0.03, jack-o'-lantern 0.03, 2.2ms
Speed: 2.7ms preprocess, 2.2ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /home/ray/Projects/ITI123-GenerativeAIDeepLear

(0.010321100917431193,
 ['ageing',
  'cataract',
  'diabetes',
  'glaucoma',
  'hypertension',
  'myopia',
  'normal',
  'other'],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,

In [8]:
best_val_f1 = 0.0
train_f1_list = []
val_f1_list = []

def macro_f1_callback(trainer):
    global best_val_f1
    global train_f1_list, val_f1_list

    epoch = trainer.epoch + 1
    model = trainer.model

    # Compute F1 on train set
    train_macro_f1, _, _, _ = compute_macro_f1(model, os.path.join(data_dir, "train"))
    train_f1_list.append(train_macro_f1)

    # Compute F1 on val set
    val_macro_f1, _, _, _ = compute_macro_f1(model, os.path.join(data_dir, "val"))
    val_f1_list.append(val_macro_f1)

    # -------------------
    # Log to W&B
    wandb.log({
        "epoch": epoch,
        "train_macro_f1": train_macro_f1,
        "val_macro_f1": val_macro_f1
    })
    print(f"[Epoch {epoch}] Train Macro-F1: {train_macro_f1:.4f} | Val Macro-F1: {val_macro_f1:.4f}")

    # -------------------
    # Save & rename best model
    if val_macro_f1 > best_val_f1:
        best_val_f1 = val_macro_f1
        best_model_path = Path(f"{result_folder}/{run_name}/best_epoch_{epoch}.pt")
        os.makedirs(best_model_path.parent, exist_ok=True)
        model.save(best_model_path)
        shutil.copy(best_model_path, final_model_path)
        print(f"Saved and renamed best model: {final_model_path}")

In [ ]:
model.add_callback("on_train_epoch_end",macro_f1_callback)
model.train(
    data=data_dir,        # folder-based (train/val)
    epochs=epochs,
    imgsz=image_size,
    batch=batch_size,
    device=0,
    workers=workers_count,
    patience=patience,
    pretrained=True,
    verbose=False)

New https://pypi.org/project/ultralytics/8.4.14 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 2050, 3769MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./ODIR-2019/YOLO/processed_512g_merged, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train10, nbs

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(range(1,epochs+1), train_f1_list, label="Train Macro-F1")
plt.plot(range(1,epochs+1), val_f1_list, label="Val Macro-F1")
plt.xlabel("Epoch")
plt.ylabel("Macro-F1")
plt.title("YOLO-CLS Macro-F1 per Epoch")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
test_f1, classes, y_true, y_pred = compute_macro_f1(model, os.path.join(data_dir, "test"))
print(f"\nTest Macro-F1: {test_f1:.4f}")

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
plt.imshow(cm, cmap="Blues")
plt.colorbar()
plt.xticks(range(len(classes)), classes, rotation=45)
plt.yticks(range(len(classes)), classes)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix (Test Set)")
plt.show()

In [ ]:
wandb.finish()